In [0]:
from pyspark.sql.functions import min, max, avg, count, col, when, filter, sum, countDistinct, round
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
spark =  SparkSession.builder\
    .appName("Data Analysis in pyspark")\
    .getOrCreate()
spark

In [0]:
emp_df = spark.read.csv('/Volumes/azure_db_ws/default/employee_data/employees.csv',
                        header=True,
                        inferSchema=True)
display(emp_df)

In [0]:
salary_df = spark.read.csv("/Volumes/azure_db_ws/default/employee_data/updated_salaries.csv",
                           header=True,
                           inferSchema=True)
display(salary_df)

In [0]:
from pyspark.sql.types import StringType, IntegerType, DateType, DoubleType

emp_df = emp_df.withColumn("emp_no", emp_df["emp_no"].cast(StringType()))
display(emp_df)

In [0]:
from pyspark.sql.functions import col, sum



In [0]:
salary_df.select(
    [sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in salary_df.columns]
).show()
# salary_df.select(
#     [sum(col(c).isNull().cast("int")).alias(c) for c in salary_df.columns]
# ).show()

In [0]:
salary_df.describe().show()


In [0]:
## Count total rows and unique employees in salary data
print(f"Total rows in salary data: {salary_df.count()}")
print(f"Unique employees in salary data: {salary_df.select('emp_no').distinct().count()}")

## Plot the salary distribution
salary_df_pandas = salary_df.toPandas()
sns.histplot(salary_df_pandas['salary'])
plt.title("Salary Distribution")
plt.show()

In [0]:
emp_df.select(
    [sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in emp_df.columns]
).show()

In [0]:
emp_df.select("first_name").distinct().count()

In [0]:
salary_summary = salary_df.groupBy("dept_no").agg(
    round(avg("salary"), 2).alias("avg(salary)"),
    max("salary").alias("max(salary)"),
    min("salary").alias("min(salary)"),
    countDistinct("emp_no").alias("distinct employee number")
)
display(salary_summary.show())

salary_summary_df = salary_summary.toPandas()
salary_summary_df.plot(kind="bar", x="dept_no", y="distinct employee number", rot=0)
